<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:

The input layer is the layer or the neural network that git the values to the neural network.  It will pass its value, multiplied with a weight to the next layer of the neural network, which is the hidden layer.  There is usually the same number of nodes in the input layer as their are features in the in data.  The input layer is the layer that gets the values from the data set and then passes them into the neural network. 

### Hidden Layer:

The hidden layer are the layers of the neural network which is not accessable from the outside of the neural network.  They recieve the values multiplied by the weights from either a previous hidden layer or from the input layer.  They will use an activation function to adjust the signal either and then send the signal to another hidden layer or to the output layer.

### Output Layer:

The output layer recieves input from the hidden layer and then will create the output.  It at times will also have a function act upon it to change the values such that the endresult is what is needed for they type of model. For example if it is a classification it will give a value that will mean a certain type of classification.

### Neuron:

A neuron is node with in a neural network. You can think of a nueron as each node in the hidden layers and the output layers, and the input layers.  

### Weight:

The weight is multiplied by the values passed to a hidden layer node.  Weights that are higher, will cause the value that is passed to the hidden layer to have a larger effect or (conncection) to the hidden node.  Values that are passed in that are multiplied with a small weight will have less of an effect on what the hidden layers will pass on to the next node. 

### Activation Function:

The activation function is a function that causes that value that the output node to have to be outputted in an expected format.  One of the common types of activation functions, for example is the sigmoid function.  This function will cause that all the values will be bounded between 1 and 0.

### Node Map:

A node map is a visual representation of the topology or the flow of the neural network.  It shows how each node will interact with the nodes around it.  

### Perceptron:

A perceptron is a single layered neural network. It will have inputs and then multiply them by the weights and then sum them up and then pass the sum into an activation function (possibly the sigmoid function) which will output a value.


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

The flow of a neural network is thus:

The inputs (the values from the data) are passed into neural network to the input layers.  There is usually one input node per feature of the data.  From the input layer the value from each node is sent to all the nodes of the next layer, which is a hidden layer.   Each node of the hidden layer will recieve a value from all the input nodes.  When the value is passed to the hidden layer, it is multiplied by a weight value and then summed up with the other values at that node that have also been multiplied by their own specific weight.  This summed up value is then added to the bias.  At each layer all the nodes will then pass their information into an activation function.  The activation function at this point is used to help determine if and what is passed on to the next layer of nodes.   This complete value is then passed on to the next layer or the neural network, which may be another hidden layer and the same process is then repeated, or it is passed to the output layer where the values are put into a transormation function.  This function will make it so the values outputed are the type expected, such as possibly bounding the values between 1 and 0.  

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
# importing numpy
import numpy as np

In [2]:
import pandas as pd
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')

In [3]:
df.head()



,x1,x2,y
0,0,0,1
1,1,0,1
2,0,1,1
3,1,1,0


In [3]:
train = df[["x1", "x2"]]
train.head()

,x1,x2
0,0,0
1,1,0
2,0,1
3,1,1


In [4]:
# creating a sigmoid function
def sigmoid_function(x):
    return 1/(1 + np.exp(-x))

In [5]:
# creating derivative of the sigmoid function
def sigmoid_derivative(x):
    sx = sigmoid_function(x)
    return (sx * (1-sx))

In [23]:
# creating the weights that will be used at the start
def create_weights(numImputs):
    np.random.seed(42)
    weights = 2 * np.random.random((numImputs,1)) -1
    return weights

In [24]:
# Looking at the weights
weights = create_weights(2)
weights

array([[-0.25091976],
       [ 0.90142861]])

In [25]:
# doing the dot product to get the weight sum
def weighted_sum(weights, input):
    weighted_sum = np.dot(input, weights)
    return weighted_sum

In [32]:
weights_sum = weighted_sum(weights, train)
weights_sum

array([[ 0.        ],
       [-0.25091976],
       [ 0.90142861],
       [ 0.65050885]])

In [33]:
# trying out the weighted_sum method
activated_output = sigmoid_function(weights_sum)
activated_output

array([[0.5       ],
       [0.43759713],
       [0.71124299],
       [0.65712512]])

In [34]:
sigmoid_function(0.90142861)

0.711242993921282

In [12]:
# making the series into a multidimensional array
def make_y_vals_correct_shape():
    y = df["y"]
    y = np.array(y)
    return  np.reshape(y, (y.shape[0], 1))


In [35]:
y = make_y_vals_correct_shape()

In [36]:
y

array([[1],
       [1],
       [1],
       [0]])

In [37]:
# calculating the difference between the correct and the 
# error

error = y - activated_output
error

array([[ 0.5       ],
       [ 0.56240287],
       [ 0.28875701],
       [-0.65712512]])

In [38]:
# now doing the adjustment of the error
# calculating the amount to adjust the weights
amount_to_adjust = error*sigmoid_derivative(weights_sum)
amount_to_adjust

array([[ 0.125     ],
       [ 0.13841065],
       [ 0.05930387],
       [-0.14805798]])

In [18]:
# now we are updating the weigts by adding the adjust amout to the weight
weights += np.dot(train.T, amount_to_adjust)
weights

array([[0.29832161],
       [0.84243177]])

In [65]:
# Creating a function that will do perceptron code it will iterate through the steps until it has set the weights and the bias
# to be able to predict with 99% accuracy.
def adjust_weights():

    # creating a set of weights to start with:
    weights = create_weights(2)
    y = make_y_vals_correct_shape()
    learning_rate = .1
    numEpochs = 15000
    # Will now the for loop for the iterations to adjust the weights
    for i in range(numEpochs):
        # Getting the activated output
        sums_of_weights = weighted_sum(weights, train) + 7
        activated_output = sigmoid_function(sums_of_weights)
        # will then the errror of the amounts
        error = y - activated_output
        # will then get the amount needed to adjust the weights
        amount_to_adjust = error*sigmoid_derivative(weights_sum) # * learning_rate
        # will now adjust the weights
        weights += np.dot(train.T, amount_to_adjust)
    
    # Will print out the weights and then the outputs
    print(f"The weights after training with {numEpochs} epochs is:\n")
    print(weights, "\n")
    # Printing out the activated outputs
    print(f"The outputs after training for {numEpochs} is:\n")
    print(activated_output)



In [66]:
# running the function above:
adjust_weights()

The weights after training with 15000 epochs is:

[[-4.56835883]
 [-4.76686051]] 

The outputs after training for 15000 is:

[[0.99908895]
 [0.9192085 ]
 [0.90318623]
 [0.08824781]]


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [67]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [72]:
from sklearn.preprocessing import MinMaxScaler, Normalizer

feats = list(diabetes)[:-1]

X = diabetes[feats]



In [101]:
diabetes["Outcome"].dtype

dtype('int64')

In [105]:
# will be doing minmax scaling
scaler = MinMaxScaler()

In [106]:
scaler.fit_transform(X)

array([[0.35294118, 0.74371859, 0.59016393, ..., 0.50074516, 0.23441503,
        0.48333333],
       [0.05882353, 0.42713568, 0.54098361, ..., 0.39642325, 0.11656704,
        0.16666667],
       [0.47058824, 0.91959799, 0.52459016, ..., 0.34724292, 0.25362938,
        0.18333333],
       ...,
       [0.29411765, 0.6080402 , 0.59016393, ..., 0.390462  , 0.07130658,
        0.15      ],
       [0.05882353, 0.63316583, 0.49180328, ..., 0.4485842 , 0.11571307,
        0.43333333],
       [0.05882353, 0.46733668, 0.57377049, ..., 0.45305514, 0.10119556,
        0.03333333]])

In [73]:
X.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31


In [130]:
##### Update this Class #####

class Perceptron:
    
    def __init__(self, niter = 10):
        self.niter = niter
    
    def __sigmoid(self, x):
        return 1/(1 + np.exp(-x))
        
    
    def __sigmoid_derivative(self, x):
        sx = self.__sigmoid(x)
        return (sx * (1-sx))

    def fit(self, X, y):
   # """Fit training data
   # X : Training vectors, X.shape : [#samples, #features]
   # y : Target values, y.shape : [#samples]
   # """
        # Will be scaling the data
        self.scaler = MinMaxScaler()
        scaled_data = self.scaler.fit_transform(X)
        num_Cols = len(X.columns.values.tolist())
        # Randomly Initialize Weights
        weights = 2 * np.random.random((num_Cols,1) ) - 1

        for i in range(self.niter):
            # Weighted sum of inputs / weights
            weight_sum = weighted_sum(weights, X)
            # Activate!
            activated_output = self.__sigmoid(X)
            
            
            # Cac error
            error = y - activated_output

            # Update the Weights
            # Getting the amount to adjust
            amount_to_adjust = error*self.__sigmoid_derivative(weight_sum)
            weights = weights + np.dot(X.T, amount_to_adjust)
        #breakpoint()
        print("The acitvated amount of the first train data after the iterations is: ", activated_output[:1], "\n")
        

    def predict(self, X):
        # Will be doing the transform of the data that is to 
        # be predicted
        scaled_data = self.scaler.transform(X)
    #"""Return class label after unit step"""
        return None

In [131]:
# Trying to run the class
p = Perceptron()

In [132]:
p.fit(X, diabetes["Outcome"])

The acitvated amount of the first train data after the iterations is:     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  \
0     0.997527      1.0            1.0            1.0      0.5  1.0   

   DiabetesPedigreeFunction  Age  
0                  0.651809  1.0   



In [ ]:
# doing a predidiction


## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?